**Author:** [Scott Sievert](https://stsievert.com)


Salmon and NEXT are two different software systems to perform the same task, adaptive sampling for the "triplet embedding" problem. Salmon and NEXT exhibit different performance when launched on EC2 as their respective documentation recommends and human responses are simulated:


Accuracy represents performance on unseen (simulated) human responses. The average number of items closer than the true nearest neighbor (NN) is perhaps a measure of embedding quality. For this target set of "alien eggs" characterized by one parameter, having this value be 1 represents a good embedding.

# Why don't Salmon and NEXT have the same performance?

What's responsible for this different performance? Fundamentally, each software system has two tasks:

* Searching for the most informative queries to ask users.
* Generating embeddings from the user's responses.

These tasks are done online as responses are being received. These systems are intertwined: the queries and query quality depend on the embedding, and the embedding depends on the responses received from each query.

So, the difference in performance must come from the question **"how do these systems differ in NEXT and Salmon?"** I will present evidence to support the following claims:

1. Salmon and NEXT's embeddings perform equally well.
2. NEXT's search—a greedy information gain search—performs better as searches efficacy increases.
    * However, a complete search presents experimental design challenges.
3. To work around those challenges, Salmon employs a more exploratory search that is a modification of NEXT's greedy search. These modifications...
     * significantly enhance Salmon's performance over NEXT's greedy search.
     * has the same "query diversity" as random sampling.

Of course, each of these claims is an empirical observation for the experiments that have been run. In these experiments, responses were submitted at an average of 1 response/sec to simulate 2 simultaneous users.[^unreal]

[^unreal]:The use of a constant response rate is slightly unrealistic; users take slightly longer to answer more difficult queries. In addition, submitting responses at a constant rate avoids some important architecture differences and allows a fair comparison.

Let's present evidence for each of these claims. When providing evidence, I will follow the data scientist workflow:

1. Launch the software on Amazon EC2 as the documentation recommends.
2. Collect (simulated) human responses.
3. Download the responses.
4. Generate an embedding offline with my own software.

This document is used to motivate changes in Salmon's search. We've also used Salmon to improve the efficacy of NEXT's search, called "NEXT proxy" below.

We've run NEXT proxy 9 different times to vary the search efficacy, and the best of those runs is shown with "best of NEXT proxies." It appears the changes made to Salmon to improve query diversity do not to degrade performance – especially relevant because Salmon's search is easier to configure.

# Embedding performance

Let's isolate the embedding performance by running NEXT and Salmon with similar searches. That is, let's configure Salmon to have a similar search to NEXT.[^arch] Then, the relevant question is **"do both software systems generate embeddings of similar quality?"**

[^arch]:There are some important architecture differences; submitting responses at a constant rate circumvent these issues.

It appears Salmon and NEXT generate equivalent embeddings, at least at this rate (1 response/sec) for (almost) equivalent searches.

# Search performance

Now, let's isolate the search performance. That is, let's use the same embedding system and vary the searches by implement different searches in Salmon.

Let's vary NEXT's search completeness (which is only practical in Salmon's system). That is, if the search evaluates $N$ queries per second, how does search performance vary? That is, let's vary `n_search` in this code:

``` python
def next_search(n_search=30):
    queries = [_random_query(n=90) for _ in range(n_search)]
    scores = [_info_gain(q) for q in queries]
    
    best_query = queries[np.argmax(scores)]
    return best_query
```

Let's vary the search by (roughly) a factor of 30, which simulates making NEXT's search 30× faster.

However, if NEXT's search is too complete, it tends to focus on a particular head:

![](figs/n-run-linear.png)

For example, here's part of the queries:

In [1]:
import pandas as pd
n_search = 30_000
df = pd.read_csv(f"salmon/io/2021-05-26-search/TSTE-n_search={n_search}-1_responses.csv.zip")
df["head"].iloc[1000:1000 + 20].to_numpy()

array([78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 25, 78, 78, 78,
       78, 78, 78])

That means the 78th head object—alien egg 524—appeared in the head position for queries 1000 through 1020:

# Salmon search validation

The issue above has motivated two changes to Salmon's search that allow for more item diversity in the queries. The search is modified to do the following:

* Randomly selecting the "head" of the query
* Randomly selecting one of the top-$k$ scoring queries for each head.

In practice, one of $k n$ high scoring queries is randomly selected if there are $n$ target items. Now, these questions are relevant:

1. Are the queries diverse enough?
1. How much do the adaptive gains depend on search length?
1. How much do the adaptive gains depend on the round-robin scheme with $k$?

## Query diversity

The "round robin" queries are certainly a lot more diverse:

This shows different search lengths and choosing the best query for each head. If the top $k$ queries are shown for the most complete search,

## Modifications

In practice, Salmon's search selects $k n$ high scoring queries with a randomly selected head.

* How important is the random selection of the head?
* How important is $k$?


### Round-robin search length

**How important is randomly rotating the head?** Unfortunately, when investigating this there's a confounding variable: the number of queries searched before randomly selecting a head.

First, let's see how much head rotation helps for a *fixed* number of queries searched:

It appears that randomly rotating the head is beneficial for any search length for both metrics, at least for these searches were the greedy search without head rotation performs decently.

As such, Salmon randomly rotates the head because it helps for relatively limited searches. Now, let's look at different search lengths against the best possible TSTE performance:

For test accuracy, it looks like all searches except the most exhaustive search beat the best greedy search. This is likely due to some difficulties with embedding this one dimensional manifold into $d=2$ dimensions. In that case, looking at the avg. number of items closer than the true NN might be better because it's a measure of true embedding quality.

### Choosing one of top-$k$ queries for each head

In practice, the Salmon search is very efficient. For $n=90$ objects embedded into $d=2$ dimensions, it can score every query. In Salmon's search, one of the top-$k$ queries is selected for each head. **How does the value of $k$ affect the search?**

Here's a graph that illustrates the performance of changing $k$ for the most complete search:

Any value of $k \le 30$ performs pretty much equivalently for accuracy, though any value of $k \le 10$ perfoms equally well for nearest neighbor distance.

This suggests a new architecture for NEXT that rotates the head randomly and searches about 400 queries.

In [5]:
n = 90
k = 10
possible_queries = (n - 1) * (n - 2) // 2
possible_queries / k

391.6

## Comparison

How does the complete Salmon search with `n_top=3` perform with the best of the 9 NEXT-like searches?

# Conclusion

We have presented evidence that suggests that NEXT's search presents experimental design challenges for exhaustive searches. To circumvent these issues, we have modified NEXT's search to include random head selection and random selection of the top-$k$ queries. Through some validation, it appears that the head rotation has the greatest affect on embedding quality, with smaller changes for search efficacy and the choice of $k$. The best adaptive performance is retained with an exhaustive search and $k=3$.

In addition, we have identified a new search architecture for NEXT.